In [1]:
#search

In [2]:
import string
import glob
import nltk
import math
import os
from collections import defaultdict
import copy
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\siddh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\siddh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\siddh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import tweepy

In [4]:
# consumer_key='fRg1ybVEYktYKmzX1yc9faJP7'
# consumer_secret='RtaJApkrJxrW4fk7SELDanXslvBztwl1qVEqIrgTAicN9YPmeA'
# access_token='1641367187230162945-B8qTOKgps8tsVeXZeuE4OJHxAanxzb'
# access_token_secret='Xrl2gjoDCukzO5hNx398FGMgJUZWIkAp9At6nzSN2oyQF'
# auth = tweepy.OAuth1UserHandler(
#    consumer_key, consumer_secret, access_token, access_token_secret
# )
# api = tweepy.API(auth)
#
# public_tweets = api.home_timeline()
# for tweet in public_tweets:
#     print(tweet.text)

In [13]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

query = "help me twitter"
tweets = []
limit = 1000


for tweet in sntwitter.TwitterSearchScraper(query).get_items():

    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount, tweet.retweetCount, tweet.hashtags])

df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'Followers', 'Likes', 'Retweets', 'Hashtags'])
print(df)


C:\Users\siddh\AppData\Local\Temp\ipykernel_18416\2664053839.py:16: DeprecatedFeatureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount, tweet.retweetCount, tweet.hashtags])
C:\Users\siddh\AppData\Local\Temp\ipykernel_18416\2664053839.py:16: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount, tweet.retweetCount, tweet.hashtags])


                         Date            User  \
0   2023-03-30 12:10:51+00:00  aishbhardwaj88   
1   2023-03-30 12:04:19+00:00  ghettocowgworl   
2   2023-03-30 12:00:22+00:00    Mike_Scully_   
3   2023-03-30 11:57:09+00:00     AlainDouzal   
4   2023-03-30 11:52:38+00:00    ophiomormous   
..                        ...             ...   
995 2023-03-27 10:08:20+00:00  bloodlineprint   
996 2023-03-27 09:48:29+00:00      floralshi_   
997 2023-03-27 09:42:00+00:00    Snowywizard_   
998 2023-03-27 09:36:31+00:00       KingThona   
999 2023-03-27 09:34:41+00:00    iheartguitar   

                                                 Tweet  Followers  Likes  \
0    Hi Twitter\n\nPlease do your magic✨and help me...       1867      2   
1    @Justoromi i thought deleting the app would he...       4480      0   
2    That's a wrap!\n\nIf you enjoyed this thread a...      17972      2   
3    Hi @elonmusk, with hardworking, I have created...       1151      3   
4    Guys, gue gabut, apakah di

In [14]:
df.to_csv('corpus.csv')

In [15]:
query = "python"
tweets = []
limit = 1000


for tweet in sntwitter.TwitterSearchScraper(query).get_items():

    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount, tweet.retweetCount, tweet.hashtags])

df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet', 'Followers', 'Likes', 'Retweets', 'Hashtags'])
print(df)

C:\Users\siddh\AppData\Local\Temp\ipykernel_18416\405132665.py:13: DeprecatedFeatureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount, tweet.retweetCount, tweet.hashtags])
C:\Users\siddh\AppData\Local\Temp\ipykernel_18416\405132665.py:13: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.user.followersCount, tweet.likeCount, tweet.retweetCount, tweet.hashtags])


                         Date             User  \
0   2023-03-30 12:20:03+00:00  RaspberryPLinux   
1   2023-03-30 12:20:01+00:00      echojobs_io   
2   2023-03-30 12:20:00+00:00       ingliguori   
3   2023-03-30 12:19:58+00:00       okdesuyone   
4   2023-03-30 12:19:57+00:00        Python59P   
..                        ...              ...   
995 2023-03-30 10:24:27+00:00          DD_Bun_   
996 2023-03-30 10:24:25+00:00            noyse   
997 2023-03-30 10:24:17+00:00        BMC_D_DEV   
998 2023-03-30 10:24:13+00:00       sscdotopen   
999 2023-03-30 10:23:58+00:00         slammllc   

                                                 Tweet  Followers  Likes  \
0    ¿Quieres aprender los conceptos básicos de la ...       1464      0   
1    ✋ Nutanix is hiring Senior Elixir Developer wi...       1292      0   
2    Exploring the #AI technology landscape: from #...      51433      0   
3    @no_X899 @paiza_learning そうよね！ そもそもaってどこで宣言された...        725      0   
4                  

In [16]:
df.to_csv('existing.csv', mode='a', index=False, header=False)